In [ ]:
import numpy as np
import pandas as pd
from viz import plot_shape
from constants import FACES
from nilearn.glm.first_level import run_glm
from nilearn.glm import compute_contrast
import matplotlib.pyplot as plt

import os
from glob import glob
from nilearn import signal

In [ ]:
v = np.load('face_derivatives/sub-01/filter/sub-01_desc-filter_shape.npy')[0]
f = FACES
figsize = (10, 8)
color = 'lightgray'
overlay = None
lims = (-.15, .15)

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(projection='3d')
coll = ax.plot_trisurf(v[:, 0], v[:, 1], v[:, 2], triangles=f, color=color, antialiased=True, cmap=None)
if overlay is not None:
    coll.set_array(overlay)

ax.view_init(90, -90)
ax.set_xlim(*lims)
ax.set_ylim(*lims)
ax.set_zlim(*lims)
ax.axis('off')

## Filtering

In [ ]:
data = np.load('face_derivatives/sub-01/interpolate/sub-01_desc-interp_shape.npy')
data = data.reshape((data.shape[0], 5023 * 3))
mu, std = data.mean(axis=0), data.std(axis=0)
data_n = (data - mu) / std

idx = 2
fig, ax = plt.subplots(nrows=2, figsize=(18, 15))
ax[0].plot(data_n[:, idx] * std[idx] + mu[idx]);

data_n_clean = signal.clean(data_n, detrend=True, t_r=0.089, low_pass=3, high_pass=0.01, standardize=False)
ax[1].plot(data_n_clean[:, idx] * std[idx] + mu[idx]);

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca_dat = pca.fit_transform(data_n_clean)

plt.figure(figsize=(15, 5))
plt.plot(pca_dat[:, 3])

## Analysis

In [ ]:
Y = np.load('face_derivatives/sub-01/filter/sub-01_desc-filter_shape.npy')
C = pd.read_csv('face_derivatives/sub-01/align/sub-01_desc-align_parameters.csv')
C['icept'] = 1

labels, results = run_glm(Y.reshape((Y.shape[0], 5023 * 3)), C.to_numpy(), noise_model='ols')
con = np.zeros(13)
con[10] = 1
con = compute_contrast(labels, results, con_val=con, contrast_type='t')

In [ ]:
ol = con.z_score().reshape((5023, 3))
ol = ol[FACES].mean(axis=1)[:, 0]
ol[np.abs(ol) < 3] = 0
plot_shape(Y[0, :, :], FACES, overlay=ol)

In [ ]:
ft = pd.read_csv('face_derivatives/sub-01/frame_times.tsv', sep='\t')
ev = pd.read_csv('exp_data/task-poc/sub-01/sub-01_task-poc_events.tsv', sep='\t')
ev = ev.loc[:, ['movement', 'trial_txt.started']].iloc[1:, :]
ev = ev.rename({'movement': 'trial_type', 'trial_txt.started': 'onset'}, axis=1)
ev['duration'] = 0.0001

from nilearn.glm.first_level import make_first_level_design_matrix
dm = make_first_level_design_matrix(
    frame_times=ft['new_frame_times'].to_numpy(),
    events=ev,
    hrf_model='fir',
    drift_model=None,
    fir_delays=range(0, 40, 1)
)
conf = pd.read_csv('face_derivatives/sub-01/align/sub-01_desc-align_parameters.csv')
conf.index = dm.index
dm = pd.concat((dm, conf), axis=1)

In [ ]:
plt.figure(figsize=(15, 4))
plt.plot(dm.iloc[:, :-13]);

In [ ]:
Y = np.load('face_derivatives/sub-01/filter/sub-01_desc-filter_shape.npy')

labels, results = run_glm(Y.reshape((Y.shape[0], 5023 * 3)), dm.to_numpy(), noise_model='ols')
con = np.zeros(dm.shape[1])
con[5] = 1
con = compute_contrast(labels, results, con_val=con, contrast_type='t')

#ol = con.z_score().reshape((5023, 3))
#ol = ol[FACES].mean(axis=1)[:, 1]
#ol[np.abs(ol) < 5] = 0
ol = results[0].r_square.reshape((5023, 3))
ol = ol[FACES].mean(axis=1)[:, 2]
plot_shape(Y[0, :, :], FACES, overlay=ol)

In [ ]:
rr = results[0]

plt.figure(figsize=(15, 3))
b = rr.theta[:40].reshape((40, 5023, 3))[:, :, 1]
b = (b - b.mean(axis=0)) / b.std(axis=0)
bp = pca.fit_transform(b)
bp = (bp - bp.mean(axis=0)) / bp.std(axis=0)
plt.plot(bp[:, :2])
plt.show()